## imports and data urls

In [1]:
import pandas as pd
import numpy as np

url1 = "https://datasets.imdbws.com/title.basics.tsv.gz"
url2 = "https://datasets.imdbws.com/title.akas.tsv.gz"
url3 = "https://datasets.imdbws.com/title.ratings.tsv.gz"

## Lets clean akas

In [2]:
%%time

# Load akas Data
akas = pd.read_csv(url2, sep='\t', low_memory=False)

CPU times: total: 2min 52s
Wall time: 4min 22s


In [3]:
%%time

akas.replace({'\\N':np.nan}, inplace=True)
akas = akas[(akas['region'] == 'US')]
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1358696 entries, 5 to 33454836
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1358696 non-null  object
 1   ordering         1358696 non-null  int64 
 2   title            1358696 non-null  object
 3   region           1358696 non-null  object
 4   language         3685 non-null     object
 5   types            964575 non-null   object
 6   attributes       45154 non-null    object
 7   isOriginalTitle  1357321 non-null  object
dtypes: int64(1), object(7)
memory usage: 93.3+ MB
CPU times: total: 1min 49s
Wall time: 2min 6s


## Lets clean title.basics dataset

In [4]:
%%time

# Load basics Data
basics = pd.read_csv(url1, sep='\t', low_memory=False)

CPU times: total: 55.2 s
Wall time: 1min 11s


In [5]:
# Replace '\N' with np.nan
basics.replace({'\\N':np.nan}, inplace=True)

In [6]:
# Drop any rows that have null values in either genre or runtimeMinutes.
basics.dropna(subset=['runtimeMinutes', 'genres'], inplace=True)

In [7]:
# Drop all rows where titleType!=Movie. Keep only titleType==Movie
is_Movie = basics['titleType'] == 'movie'
basics = basics[is_Movie]

In [8]:
# Drop all rows where the start year is not between 2000-2022. Keep startYear 2000-2022

# We first need to change the column type to something numeric.
print("\nData Type before conversion : ", basics['startYear'].dtype)
basics['startYear'] = pd.to_numeric(basics['startYear'])
print("\nData Type after conversion : ", basics['startYear'].dtype)

#Now we can make and apply our filters
filter2000 = basics['startYear'] >= 2000 
filter2022 = basics['startYear'] <=2022
basics = basics[filter2000 & filter2022]


Data Type before conversion :  object

Data Type after conversion :  float64


In [9]:
# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

In [10]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers_basics = basics['tconst'].isin(akas['titleId'])
basics = basics[keepers_basics]

basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82902 entries, 34792 to 9289180
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          82902 non-null  object 
 1   titleType       82902 non-null  object 
 2   primaryTitle    82902 non-null  object 
 3   originalTitle   82902 non-null  object 
 4   isAdult         82902 non-null  object 
 5   startYear       82902 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  82902 non-null  object 
 8   genres          82902 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.3+ MB


## Lets clean ratings

In [11]:
%%time

# Load ratings Data
ratings = pd.read_csv(url3, sep='\t', low_memory=False)

CPU times: total: 2.33 s
Wall time: 3.33 s


In [12]:
ratings.replace({'\\N':np.nan}, inplace=True)
keepers = ratings['tconst'].isin(akas['titleId'])
ratings = ratings[keepers]

ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 471722 entries, 0 to 1237847
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         471722 non-null  object 
 1   averageRating  471722 non-null  float64
 2   numVotes       471722 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 14.4+ MB


## save cleaned files in Data folder

In [13]:
# Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)
print("Done!")

Done!
